# Publish an entire folder as an OER "book"

In [1]:
!rm -rf bookfiles

In [2]:
!mkdir -p bookfiles

In [3]:
from pathlib import Path
from jupyterinstruct import InstructorNotebook as inb
from jupyterinstruct.nbfilename import nbfilename
from jupyterinstruct.nbvalidate import validate
from jupyterinstruct import webtools 
from nbconvert.preprocessors import ExecutePreprocessor

<IPython.core.display.Javascript object>

In [4]:
import thisbook

tags = thisbook.tags()

In [5]:
# jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace $nbfile
# jupyter nbconvert --execute --no-prompt --allow-errors --to html $nbfile
# #	jupyter nbconvert $nbfile --to html
# cp $nbfile $webdir 
# newfile=`basename $nbfile .ipynb`
# cp ${newfile}.html $webdir 

In [6]:
def makechapter(filename,tags):
    print(filename)
    nb = inb.InstructorNB(filename=filename)

    nb.removebefore('ENDHEADER')
    nb.removeafter('STARTFOOTER')
    nb.headerfooter('Book-Header.ipynb', 'Book-Footer.ipynb')
    nb.removecells('###MSU###')
    studentfile = nb.makestudent(tags,studentfolder="./bookfiles/")
    nb.writenotebook(studentfile)
    validate(studentfile)
   
    #done = webtools.publish2folder(studentfile, website_folder="./bookfiles/")
    #display(done)
    return studentfile

In [7]:
#Make a temporary folder and convert all Instructor
#notebooks to student notebooks for testing

directory = Path('..')

allfiles = sorted(directory.glob('*.ipynb'))
allfiles.sort
#Make student versions
for filename in allfiles:
    nbfile = nbfilename(filename)
    if not 'Quiz' in str(nbfile): # Skip quiz prep
        if nbfile.isInstructor: # Only process INSTRUCTOR notebooks
            if nbfile.prefix.isdigit(): # Only process numrical notebooks
                filename = makechapter(filename, tags)

../0-Introduction_INSTRUCTOR.ipynb
Myfilename ../0-Introduction_INSTRUCTOR.ipynb


0-Introduction.ipynb


Validating Notebook ./bookfiles/0-Introduction.ipynb
   ERROR: Image LINK not found - #Undergraduate-Learning-Assistance
../00-Class_Prep_Checklist_INSTRUCTOR.ipynb
Myfilename ../00-Class_Prep_Checklist_INSTRUCTOR.ipynb


00-Class_Prep_Checklist.ipynb


Validating Notebook ./bookfiles/00-Class_Prep_Checklist.ipynb
   ERROR: File Doesn't Exist - 01-Python_Packages
   ERROR: File Doesn't Exist - 02--Vectors_pre-class-assignment
../0000--Python_Linear_Algebra_Packages_pre-class-assignment-INSTRUCTOR.ipynb
Myfilename ../0000--Python_Linear_Algebra_Packages_pre-class-assignment-INSTRUCTOR.ipynb


...

Validating Notebook ./bookfiles/20-LSF_Practice_in-class-assignment.ipynb

   ERROR: No Alt text in image - data:image/png;base64,iVBORw0KGgoAAAANSUhE..
   ERROR: No Alt text in image - data:image/png;base64,iVBORw0KGgoAAAANSUhE..
   ERROR: No Alt text in image - data:image/png;base64,iVBORw0KGgoAAAANSUhE..
   ERROR: No Alt text in image - data:image/png;base64,iVBORw0KGgoAAAANSUhE..
   ERROR: No Alt text in image - data:image/png;base64,iVBORw0KGgoAAAANSUhE..
   ERROR: No Alt text in image - data:image/png;base64,iVBORw0KGgoAAAANSUhE..
../21--Review_pre-class-assignment-INSTRUCTOR.ipynb
Myfilename ../21--Review_pre-class-assignment-INSTRUCTOR.ipynb


21--Review_pre-class-assignment.ipynb


Validating Notebook ./bookfiles/21--Review_pre-class-assignment.ipynb
../21-Review_in-class-assignment-INSTRUCTOR.ipynb
Myfilename ../21-Review_in-class-assignment-INSTRUCTOR.ipynb


21-Review_in-class-assignment.ipynb


Validating Notebook ./bookfiles/21-Review_in-class-assignment.ipynb
   - ERROR: Extra Tag #TODO#



In [8]:
#makechapter(Path('../00--Introduction_INSTRUCTOR.ipynb'),tags);

In [9]:
fid = open('./bookfiles/_toc.yml', 'w')
directory = Path('./bookfiles/')

allfiles = sorted(directory.glob('*.ipynb'))

for ipynb in allfiles:
    if not '0000' in str(ipynb):
        print(ipynb.name)
        fid.write(f'- file: {ipynb.name}\n')
    
for ipynb in allfiles:
    if '0000' in str(ipynb):
        print(ipynb.name)
        fid.write(f'- file: {ipynb.name}\n')
    
fid.close()

0-Introduction.ipynb
00-Class_Prep_Checklist.ipynb
01-CMSE314_introduction_in-class-assignment.ipynb
02--Vectors_pre-class-assignment.ipynb
02-Vectors_in-class-assignment.ipynb
03--Linear_Equations_pre-class-assignment.ipynb
03-Linear_Equations_in-class-assignment.ipynb
04--Gauss_Jordan_pre-class-assignment.ipynb
04-Gauss_Jordan_in-class-assignment.ipynb
05--Gauss_Jordan2_pre-class-assignment.ipynb
05-Gauss_Jordan2_in-class-assignment.ipynb
06--Mechanics_pre-class-assignment.ipynb
06-Mechanics_in-class-assignment.ipynb
07--Transformations_pre-class-assignment.ipynb
07-Transformations_in-class-assignment.ipynb
08--Robotics_pre-class-assignment.ipynb
08-Robotics_in-class-assignment.ipynb
09--Determinants_pre-class-assignment.ipynb
09-Determinats_in-class-assignment.ipynb
10--Eigenproblems_pre-class-assignment.ipynb
10-Eigenproblems_in-class-assignment.ipynb
11--Vector_Spaces_pre-class-assignment.ipynb
11-Vector_Spaces_in-class-assignment.ipynb
12--Change_Basis_pre-class-assignment.ipynb


In [10]:
!cp Cover.jpg ./bookfiles/
!cp _config.yml ./bookfiles/

In [11]:
!jupyter-book build ./bookfiles/

Running Jupyter-Book v0.10.2
Source Folder: /Users/colbrydi/REPOS/MTH314_Textbook/MakeTextbook/bookfiles
Config Path: /Users/colbrydi/REPOS/MTH314_Textbook/MakeTextbook/bookfiles/_config.yml
Output Path: /Users/colbrydi/REPOS/MTH314_Textbook/MakeTextbook/bookfiles/_build/html
Running Sphinx v3.5.4
making output directory... done
myst v0.13.7: MdParserConfig(renderer='sphinx', commonmark_only=False, dmath_allow_labels=True, dmath_allow_space=True, dmath_allow_digits=True, update_mathjax=True, enable_extensions=['colon_fence', 'dollarmath', 'linkify', 'substitution'], disable_syntax=[], url_schemes=['mailto', 'http', 'https'], heading_anchors=None, html_meta=[], footnote_transition=True, substitutions=[], sub_delimiters=['{', '}'])
building [mo]: targets for 0 po files that are out of date
building [html]: targets for 45 source files that are out of date
updating environment: [new config] 45 added, 0 changed, 0 removed
Executing: 0-Introduction in: /Users/colbrydi/REPOS/MTH314_Textbook/M

In [14]:
!mv ./bookfiles/_build/html ./docs

In [15]:
!open ./docs/index.html
